import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
import findspark
findspark.init()
import pandas as pd

create local SparkSession

In [5]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('PySpark_Basics') \
                    .getOrCreate()

read csv with inferschema

In [6]:
df = spark.read.csv("ds_salaries.csv")
df.show()

+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| _c0|      _c1|             _c2|            _c3|                 _c4|     _c5|            _c6|          _c7|               _c8|         _c9|            _c10|        _c11|
+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|null|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
|   0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|   1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [7]:
df = spark.read.csv("ds_salaries.csv")

write schema of scv on screen

In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



create schema of this scv

In [9]:
schema = StructType([
        StructField("employee_id", IntegerType()),
        StructField("work_year", IntegerType()),
        StructField("experience_level", StringType()),
        StructField("employment_type", StringType()),
        StructField("job_title", StringType()),
        StructField("salary", IntegerType()),
        StructField("salary_currency", StringType()),
        StructField("salary_in_usd", IntegerType()),
        StructField("employee_residence",StringType()),
        StructField("remote_ratio", IntegerType()),
        StructField("company_location", StringType()),
        StructField("company_size", StringType())
    ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [10]:
df = spark.read \
  .option("header", True) \
  .schema(schema) \
  .csv("ds_salaries.csv")
df.show()

+-----------+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|employee_id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+-----------+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|          0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|          1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|          2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|   

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [11]:
df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [12]:
df.show()


+-----------+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|employee_id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+-----------+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|          0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|          1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|          2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|   

print data in dataframe using display(df.toPandas())

In [13]:
display(df.toPandas())

,employee_id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [14]:
df_job_title = df.dropDuplicates(['job_title'])
print("Distinct count: "+str(df_job_title.count()))

Distinct count: 50


print all rows from df_job_titles without truncating jobs

In [15]:
df_job_title.show(truncate=False)

+-----------+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|employee_id|work_year|experience_level|employment_type|job_title                         |salary |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+-----------+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|77         |2021     |MI              |PT             |3D Computer Vision Researcher     |400000 |INR            |5409         |IN                |50          |IN              |M           |
|52         |2020     |EN              |FT             |AI Scientist                      |300000 |DKK            |45896        |DK                |50          |DK              |S           |
|344        |2022     |EX              |

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [16]:
df_analytic = df.groupby('job_title').agg(max('salary_in_usd').alias("max_salary"), avg('salary_in_usd').alias("avg_salary"), min('salary_in_usd').alias("min_salary"),)

print all rows from df_analytic without trancating jobs

In [17]:
df_analytic.show(truncate = False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [18]:
windowSpec  = Window.orderBy(col("avg_salary").desc())
df_analytic = df_analytic.withColumn("row_id", row_number().over(windowSpec))

print all data from df_analytic

In [19]:
df_analytic.show(truncate = False)

+----------------------------------+----------+------------------+----------+------+
|job_title                         |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------+----------+------------------+----------+------+
|Data Analytics Lead               |405000    |405000.0          |405000    |1     |
|Principal Data Engineer           |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst            |450000    |275000.0          |100000    |3     |
|Principal Data Scientist          |416000    |215242.42857142858|148261    |4     |
|Director of Data Science          |325000    |195074.0          |130026    |5     |
|Data Architect                    |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist            |380000    |175655.0          |54238     |7     |
|Analytics Engineer                |205300    |175000.0          |135000    |8     |
|Data Specialist                   |165000    |165000.0          

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [20]:
df_analytic = df_analytic.select('row_id', 'job_title', 'max_salary', 'avg_salary', 'min_salary')

print df_analytic now

In [21]:
df_analytic.show()

+------+--------------------+----------+------------------+----------+
|row_id|           job_title|max_salary|        avg_salary|min_salary|
+------+--------------------+----------+------------------+----------+
|     1| Data Analytics Lead|    405000|          405000.0|    405000|
|     2|Principal Data En...|    600000| 328333.3333333333|    185000|
|     3|Financial Data An...|    450000|          275000.0|    100000|
|     4|Principal Data Sc...|    416000|215242.42857142858|    148261|
|     5|Director of Data ...|    325000|          195074.0|    130026|
|     6|      Data Architect|    266400| 177873.9090909091|     90700|
|     7|Applied Data Scie...|    380000|          175655.0|     54238|
|     8|  Analytics Engineer|    205300|          175000.0|    135000|
|     9|     Data Specialist|    165000|          165000.0|    165000|
|    10|        Head of Data|    235000|          160162.6|     32974|
|    11|Machine Learning ...|    260000|          158412.5|     12000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [22]:
df_exp_lvl = df.groupby('experience_level').agg(max('salary_in_usd').alias("biggest_salary"))

print here df_exp_lvl

In [23]:
df_exp_lvl.show(truncate = False)

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|EX              |600000        |
|MI              |450000        |
|EN              |250000        |
|SE              |412000        |
+----------------+--------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: employee_id, experience_level, biggest_salary, employee_residence

In [24]:
df_exp_lvl = df_exp_lvl.withColumnRenamed('experience_level','exp_lvl')
df_exp_lvl.show(5)

+-------+--------------+
|exp_lvl|biggest_salary|
+-------+--------------+
|     EX|        600000|
|     MI|        450000|
|     EN|        250000|
|     SE|        412000|
+-------+--------------+



In [25]:
df_exp = df.join(df_exp_lvl, df.experience_level ==  df_exp_lvl.exp_lvl)
df_exp.show(5)

+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+-------+--------------+
|employee_id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|exp_lvl|biggest_salary|
+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+-------+--------------+
|          0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|     MI|        450000|
|          1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|     SE|        412000|


In [26]:
df_best = df_exp.filter(df_exp.salary_in_usd == df_exp.biggest_salary)

In [27]:
df_best = df_best.select('employee_id', 'exp_lvl', 'biggest_salary', 'employee_residence')

print df_best

In [28]:
df_best.show(truncate = False)

+-----------+-------+--------------+------------------+
|employee_id|exp_lvl|biggest_salary|employee_residence|
+-----------+-------+--------------+------------------+
|33         |MI     |450000        |US                |
|37         |EN     |250000        |US                |
|63         |SE     |412000        |US                |
|97         |MI     |450000        |US                |
|252        |EX     |600000        |US                |
+-----------+-------+--------------+------------------+



drop duplicates if exist by experience_level

In [29]:
df_best = df_best.dropDuplicates(['exp_lvl'])

print df_best

In [280]:
df_best.show(5)

+-----------+-------+--------------+------------------+
|employee_id|exp_lvl|biggest_salary|employee_residence|
+-----------+-------+--------------+------------------+
|         37|     EN|        250000|                US|
|        252|     EX|        600000|                US|
|         33|     MI|        450000|                US|
|         63|     SE|        412000|                US|
+-----------+-------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [30]:
df_new_best = df_best.select('exp_lvl', 'biggest_salary', 'employee_residence')
df_new_best = df_new_best.withColumn('level', when(df_new_best.exp_lvl == "MI","middle") \
                                            .when(df_new_best.exp_lvl == "SE","senior"))


print df_new_best

In [47]:
type(df_new_best)

pyspark.sql.dataframe.DataFrame

write df_new_best like file.csv and load then it to df_final

In [57]:
#wrong df_new_best.to_csv("file.csv", index=False)
#wrong df_new_best.repartition(1).write.format('com.databricks.spark.csv').save('haha',header = 'true')
df_new_best.toPandas().to_csv("file.csv")

In [58]:
df_final = spark.read .format("csv").option("header", True).option("inferSchema", True).option("path", "file.csv").load()

print df_final

In [59]:
df_final.show()

+---+-------+--------------+------------------+------+
|_c0|exp_lvl|biggest_salary|employee_residence| level|
+---+-------+--------------+------------------+------+
|  0|     EN|        250000|                US|  null|
|  1|     EX|        600000|                US|  null|
|  2|     MI|        450000|                US|middle|
|  3|     SE|        412000|                US|senior|
+---+-------+--------------+------------------+------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [78]:
df_final = df_final.na.drop()
df_final.show()

+---+--------------+------------------+------+
|_c0|biggest_salary|employee_residence| level|
+---+--------------+------------------+------+
|  2|        450000|                US|middle|
|  3|        412000|                US|senior|
+---+--------------+------------------+------+



In [79]:
df_f = df_final.join(df, (df.salary_in_usd ==  df_final.biggest_salary) & (df.employee_residence == df_final.employee_residence))

print df_final

In [80]:
df_f.show()

+---+--------------+------------------+------+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|biggest_salary|employee_residence| level|employee_id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+--------------+------------------+------+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  2|        450000|                US|middle|         33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
|  3|        412000|                US|senior|         63|     2020|              SE|             FT|      Data 

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [87]:
biggest_salary_in_usd = df_f.show()

+---+--------------+------------------+------+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|biggest_salary|employee_residence| level|employee_id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+--------------+------------------+------+-----------+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  2|        450000|                US|middle|         33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
|  3|        412000|                US|senior|         63|     2020|              SE|             FT|      Data 

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system